<a href="https://colab.research.google.com/github/MohsenJadidi/ISLR/blob/master/Logistic_Regression_in_TensorFlow_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression in TensorFlow 2.0 (MNIST Data set)

Author: https://theclickreader.com/courses/tensorflow-2-for-deep-learning/lessons/logistic-regression-with-tensorflow/

Modify by: MohammadMohsen Jadidi | email : mohsenjadidi@aut.ac.ir

## Read Data set

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

# Load train and test data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Converting data to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

# Batching 
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(256).prefetch(1)

11493376/11490434 [==============================] - 0s 0us/step


## Create Model

In [2]:
class Model:
    def __init__(self):
        self.W = tf.Variable(tf.ones([28*28, 10]), name="weight")
        self.b = tf.Variable(tf.zeros([10]), name="bias")
    def __call__(self, x):
        return tf.nn.softmax(tf.matmul(x, self.W) + self.b)
    
   
def loss(y_pred, y_true):
    # Encode label to a one hot vector
    y_true = tf.one_hot(y_true, depth=10)
    
    # Clip prediction values to avoid log(0) error
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    
    # Compute cross-entropy
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),1))
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(lr = 0.1)
    

def train(model, x, y):
    with tf.GradientTape() as t:
        pred = model(x)
        current_loss = loss(pred, y)
    # Compute gradients
    gradients = t.gradient(current_loss, [model.W, model.b])
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [model.W, model.b]))